In [6]:
%pip install -q \
    fuzzywuzzy \
    easyocr \
    pandas \
    requests

Note: you may need to restart the kernel to use updated packages.


In [118]:
# seed reference items using api

# import from csv
import pandas as pd
import requests

referenceitems_df = pd.read_csv('referenceitems.csv')

def writeToEndpoint(endpoint, json):
    url = f'http://localhost:3001/api/{endpoint}'
    response = requests.post(url, json=json)
    if response.status_code == 200:
        print(f'Reference item successfully written. Response:\n{response.text}')
    else:
        print(f'Failed to write reference item. Response:\n{response.text}')
    return response

for index, row in referenceitems_df.iterrows():
    response = writeToEndpoint('referenceItem', {
        "name": row['name'],
        "quantity": row['quantity'],
        "unitOfMeasure": row['unitofmeasure'],
        "price": row['price'],
        "pricePerWeight": row['priceperweight'],
        "referenceUrl": row['referenceurl']
    })
    new_referenceitem_id = response.json()['id']
    referenceitems_df.at[index, 'id'] = int(new_referenceitem_id)
referenceitems_df['id'] = referenceitems_df['id'].astype(int)
referenceitems_df.set_index('id', inplace=True)
# referenceitems_df = referenceitems_df[['id', 'name', 'quantity', 'unitofmeasure', 'price', 'priceperweight', 'referenceurl']]
referenceitems_df

# test
# writeReferenceItem({
#     "name": "Sample item",
#     "quantity": 10,
#     "unitOfMeasure": "g",
#     "price": 100,
#     "pricePerWeight": 10,
#     "referenceUrl": "test.com"
# })

# import from queryclassification.csv to panda dataframe
queryclassification_df = pd.read_csv('queryclassification.csv')
queryclassification_df.reset_index(drop=True, inplace=True)
queryclassification_df['id'] = queryclassification_df.index
queryclassification_df.set_index('id', inplace=True)
queryclassification_df

temp_referenceItem_map = referenceitems_df
temp_referenceItem_map['id'] = temp_referenceItem_map.index
temp_referenceItem_map

queryclassification_df['referenceItemId'] = queryclassification_df['referenceItem'].map(temp_referenceItem_map.set_index('name')['id'].to_dict())
try:
    queryclassification_df['referenceItemId'] = queryclassification_df['referenceItemId'].astype(int)
except:
    print('Failed to convert referenceItemId to int')
    print(queryclassification_df[queryclassification_df['referenceItemId'].isnull()])
queryclassification_df.drop(columns=['referenceItem'], inplace=True)
queryclassification_df

Reference item successfully written. Response:
{"id":808,"name":"BBQ Sauce","quantity":455,"unitOfMeasure":"g","price":2.47,"pricePerWeight":0.005429,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-08-01T01:36:00.213Z","updatedAt":"2024-08-01T01:36:00.213Z"}
Reference item successfully written. Response:
{"id":809,"name":"Beans","quantity":540,"unitOfMeasure":"g","price":1.37,"pricePerWeight":0.002537,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-08-01T01:36:00.219Z","updatedAt":"2024-08-01T01:36:00.219Z"}
Reference item successfully written. Response:
{"id":810,"name":"Bouillon cubes","quantity":80,"unitOfMeasure":"g","price":1.14,"pricePerWeight":0.01425,"referenceUrl":"https://walmart.ca/#missing","createdAt":"2024-08-01T01:36:00.226Z","updatedAt":"2024-08-01T01:36:00.226Z"}
Reference item successfully written. Response:
{"id":811,"name":"Candy","quantity":150,"unitOfMeasure":"g","price":1,"pricePerWeight":0.006667,"referenceUrl":"https://walmart.ca/#

,query,referenceItemId
id,,
0,3x KIND Oats & Honey Granola with Toasted Coco...,821
1,Annie's Homegrown Gluten-Free Rice Pasta & Che...,826
2,Annie's Homegrown Gluten-Free Rice Pasta White...,826
3,Annies rice pasta and white cheddar,826
4,Baldstreet all beef frankfurters,847
...,...,...
169,Mac & Cheese,826
170,Caramel Chocolate Chip Cookie Bar,841
171,Purest White Bread,839


In [41]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import easyocr
import pandas as pd

reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

def ocrImage(croppedImageID):
    image_url = get_image_url(croppedImageID)
    receipt_strings = reader.readtext(image_url)

    # guard clause
    if len(receipt_strings) < 1:
        return # no text found
    
    # ... do the things

    # "receiptTexts": [
    #                     {
    #                         "id": "123",
    #                         "text": "Item 1",
    #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
    #                         "imageFileId": "123"
    #                     },
    #                     {
    #                         "id": "124",
    #                         "text": "Item 2",
    #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
    #                         "imageFileId": "123"
    #                     }
    #                 ]

    # transformation
    receipt_strings_df = pd.DataFrame(receipt_strings, columns=['boundingBox', 'text', 'confidence'])
    receipt_strings_df.drop(columns=['confidence'], inplace=True)
    receipt_strings_df['imageFileId'] = croppedImageID

    # push to API
    json_data = receipt_strings_df.to_json(orient='records')
    response = writeToEndpoint('receiptText', json_data)

    return response

# import from queryclassification.csv to panda dataframe
queryclassification_df = pd.read_csv('queryclassification.csv')


choices = { # static for now is ok because parsing is done in notebook, api is just crud
    "data": [
        {
            "id": "123",
            "query": "Schar Gluten Free Hamburger Buns",
            "referenceItemId": "123"
        },
        {
            "id": "124",
            "query": "Promise Brioche Buns",
            "referenceItemId": "123"
        }
    ]
}
choices_dict = {val['id']: val['query'] for val in choices['data']}
def get_reference_item_id(best_match):
    for choice in choices:
        if choice['query'] == best_match:
            return choice['referenceItemId']

# consider global API function: query or mutation
# async def api_call(queryormutation, variable_values):
    # try:
    #     data = await client.execute_async(queryormutation, variable_values=variable_values)
    #     print(data)
    # except Exception as e:
    #     print(f"Query failed: {e}")

def fetch_receipt(receiptID):
# TODO: implement
# async def fetch_receipt_details():
    # try:
    #     data = await client.execute_async(query, variable_values={"receiptID": receiptID})
    #     print(data)
    # except Exception as e:
    #     print(f"Query failed: {e}")
    return {
        "data": {
            "receipt": {
                # "vendor": {
                #     "id": "123",
                #     "name": "Vendor Name"
                # },
                # "images": [
                #     {
                #         "id": "123",
                #         "filename": "IMG_5903.jpg"
                #     }
                # ],
                # "receiptTexts": [
                #                     {
                #                         "id": "123",
                #                         "text": "Item 1",
                #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
                #                         "imageFileId": "123"
                #                     },
                #                     {
                #                         "id": "124",
                #                         "text": "Item 2",
                #                         "boundingBox": "{'x':0,'y':0,'width':100,'height':20}", 
                #                         "imageFileId": "123"
                #                     }
                #                 ]
            }
        }
    }

def parseTextForEligibleExpenses(receipt_id):
    receipt = fetch_receipt(receipt_id)
    receipt_texts = receipt['data']['receipt']['receiptTexts']
    threshold = 80

    # TODO: some clean up before look up
        # trim numbers in the beginning of the string, drop too short alphanumeric strings
            # search_query = re.sub(r'^\d{1,10}', '', search_query)
            # # if len(search_query) > 2:
        # if search_query contains at least one digit or letter
            # if any(char.isalnum() for char in search_query):
        # regex out 1 to 10 digits in the beginning of the string for sku codes in store receipts
    
    # for now choices are static
    # choices = get_choices() # receipt_text, receipt_vendor)
    # TODO: clean up choices too 
        # avoid strict substring matches
            # len(search_query)/len(best_match) < 0.4
        # here I could also optimize for a given vendor if available
            # receipt_vendor = receipt['data']['receipt']['vendor']['name']

    for receipt_text in receipt_texts:
        receipt_text_id = receipt_text['id']
        receipt_text = receipt_text['text']
        _, best_score, best_match_id = process.extractOne(receipt_text, choices_dict, scorer=fuzz.token_set_ratio)
        if best_score >= threshold:
            # found a decent match
            # get corresponding referenceItem 
            reference_item_id = choices[best_match_id]['referenceItemId']
            # create eligible expense
            writeToEndpoint('expense',{
                    'receiptTextId': receipt_text_id,
                    'referenceItemId': reference_item_id,
                    'receiptId': receipt_id,
                    'priceEach': 0, # TODO: find price
                    'quantiy': 1, # TODO: find quantity
                }
            )

def main_single_receipt(images): # can test with one image, but could be multiple images per receipt (or PDF file, etc)
    # direct
    # new_images = upload_images(images) # assume return ids

    # "chron/service-style", UI says processing in background
    # cropped_images = crop_images(new_images) # assume return ids
    # skip for now
    cropped_images = [123]
    receiptTexts = ocrImage(cropped_images[0]) # TODO: for loop images
    # ... need to match with reference items
    # createReceipt(receiptTexts)
    return 123 # receiptID

main_single_receipt('_') #IMG_4553.jpg
# ./store/optimized_images/1.jpg ... url idea

# def main_bulk(folder):
#     # images = os.listdir(folder)
#     # for image in images:
#     #     main_single_receipt(image) # maybe async
#     pass

Would need to write receipt texts: [{"boundingBox":[[141,101],[619,101],[619,205],[141,205]],"text":"IFRESHCO","imageFileId":123},{"boundingBox":[[87,232],[594,232],[594,280],[87,280]],"text":"CARLETON PLACE LANSDOWNE FRESHCO _","imageFileId":123},{"boundingBox":[[179,270],[499,270],[499,310],[179,310]],"text":"110 Lansdowne Avenue","imageFileId":123},{"boundingBox":[[103,304],[237,304],[237,340],[103,340]],"text":"Car Ieton","imageFileId":123},{"boundingBox":[[245,309],[265,309],[265,335],[245,335]],"text":"P","imageFileId":123},{"boundingBox":[[273,301],[377,301],[377,337],[273,337]],"text":"ace ,ON","imageFileId":123},{"boundingBox":[[395,295],[587,295],[587,334],[395,334]],"text":"613-253-6141","imageFileId":123},{"boundingBox":[[178,322],[513,322],[513,372],[178,372]],"text":"HST# 844058206 RTOOO1","imageFileId":123},{"boundingBox":[[45,398],[203,398],[203,435],[45,435]],"text":"Served bv:","imageFileId":123},{"boundingBox":[[217,399],[269,399],[269,431],[217,431]],"text":"D M","i

123

,query,referenceItem,referenceItemId
id,,,
0,3x KIND Oats & Honey Granola with Toasted Coco...,Granola,596
1,Annie's Homegrown Gluten-Free Rice Pasta & Che...,Mac and cheese,601
2,Annie's Homegrown Gluten-Free Rice Pasta White...,Mac and cheese,601
3,Annies rice pasta and white cheddar,Mac and cheese,601
4,Baldstreet all beef frankfurters,Hot dogs saussages,622
...,...,...,...
169,Mac & Cheese,Mac and cheese,601
170,Caramel Chocolate Chip Cookie Bar,Cookies,616
171,Purest White Bread,Bread,614
